# Introduction

The following notebook contains all the codes, graphs and simulations written in Julia version 1.9.0.

This notebook is part of the Master Thesis "Modeling and dynamical analysis of cortical network activity in semantic priming". It focuses on simulating the Prime-Target network encoded in semantic memory. More specifically, a study of the dynamics of the network made of 2 populations of **excitatory** (E) neurons coding for the target and the prime items, and of a population of **inhibitory** (I) neurons regulating the network is performed. The whole network with these three populations forms what is called a *Wilson-Cowan model* (WC).

Moreover, the Master Thesis is based on the paper from N. Brunel and F. Lavigne (2009) "Semantic Priming in a Cortical Network Model" (Journal of Cognitive Neuroscience, Vol 21, Issue 12, pp. 2300-2319)

# Coding part
## 0.1 Settings

In [ ]:
#= =====================================
# Install useful packages if needed
using Pkg

Pkg.add("Parameters")

# Plotting packages
Pkg.add("Plots")
Pkg.add("GLMakie")
Pkg.add("LaTeXStrings")

# Mathematical operation packages
Pkg.add("LinearAlgebra")
Pkg.add("QuadGK")
Pkg.add("Statistics")
Pkg.add("OrdinaryDiffEq")
Pkg.add("NLsolve")
Pkg.add("Printf")
Pkg.add("Interpolations")
===================================== =#

In [ ]:
# Importing useful packages
using Parameters

# Plotting packages
using Plots, LaTeXStrings
using Plots.PlotMeasures
import Plots.PlotMeasures.w as width

colors = [:CornflowerBlue,:DarkOrange,:Magenta,:DarkGray,:Firebrick,
          :LightSeaGreen,:violet,:Gold,:MediumBlue,:DeepPink,:BurlyWood,:black,
          :Red,:Cyan,:teal,:Coral,:MediumTurquoise, :MediumVioletRed,:Indigo,
          :Peru,:RoyalBlue,:PaleVioletRed, :SeaGreen,:Magenta,:Marroon,
          :Crimson, :IndianRed, :DarkCyan, :Darkviolet]

# Mathematical operation packages
using LinearAlgebra, QuadGK, Statistics, NLsolve
using OrdinaryDiffEq
using Interpolations

# Debugging
using Printf

## 0.2 Helper functions

In [ ]:
@with_kw struct default_pars
    #= ===============================================================
        Structure used to get default parameters whenever needed.
    =============================================================== =#
    
    # General parameters
    τ = 10.             # Timescale of the E populations rate dynamics [ms]
    p_i = 2             # Number of items within a group
    p_g = 1             # Number of groups
    p = 2               # Number of selective populations. (p_g*p_i)
                        # if non overlapping groups


    # Synaptic connection strengths/efficacy
    a = 0.02            # Association stength
    JE = 3.             # Average excitatory synaptic strength
    JS = 0.02           # Strength of synaptic potentiation (default in B&L: 3.65)
    JI = JE             # Inhibitory synaptic efficacy
    J1 = JE + JS        # Intrapopulation efficacy
    
    J0 = JE - JS*(a*(p_i-1)+1)/((1-a)*(p_i-1)+p-p_i)   # Synaptic strength between
                                                       # non associated items.
    Ja = J0 + a*(J1-J0) # Synaptic strength between associated items
    
    w1 = (J1-JI)/p
    w2_assoc = (Ja-JI)/p
    w2_nassoc = (J0-JI)/p
    W_assoc = [w1 w2_assoc;w2_assoc w1]
    W_nassoc = [w1 w2_nassoc;w2_nassoc w1]

    
    # External inputs
    I_sel_T = 0.15        # Selective external current [mA] for target
    Iapp_T = t->I_sel_T   # Applied current to target variable.
    
    I_sel_P = I_sel_T     # Selective external current [mA] for prime.
    Iapp_P = t->I_sel_P   # Applied current to prime variable.
    
    Iapp = t->[Iapp_P(t);Iapp_T(t)]
    
    # Non selective external current input [mA]. Set to obtain a 
    # spontaneous activity of 5 Hz with JS = 0.02 = a and p_i=2
    # Chosen such that I_ext = Phi^-1(r_spont) - (w1+w2)*r_spont
    # with w1 = (J1-JI)/p and w2 ∈ {(Ja-JI)/p; (J0-JI)/p}
    I_ext_assoc = -0.181124
    I_ext_nassoc = -0.179083
    

    # Simulation case flag
    simu_assoc = true   # Boolean flag for case simulation selection.
    
    
    # Rate contstants
    r_spont = 5.        # Spontaneous activity [Hz] of each population.
    threshold = 27.     # Threshold for reaction time [Hz].
    
    
    # Simulation parameters
    T = 100.            # Total duration of simulation [ms]
    dt = .5             # Simulation time step [ms]
    r_init = [3.; 3.]   # Initial value of prime and target rate [Hz]

    # Vector of discretized time points [ms]
    range_t = 0:dt:T-dt
    
end

In [ ]:
function Phi(x, σ=0.5, τ_memb=10.)
    #= ===============================================================
        Population transfer (or activation) function from B&L paper
    
    Arg:
    x      : The synaptic input current that the population receives 
             (Float64).
    
    σ      : Standard deviation (or square root of the variance) of
             the fluctuations (or noise).
             Note that here, a Gaussian White Noise (GWN) is used.
    
    τ_memb : Time constant of the cell membrane [msec] !.

    
    Returns:
    phi      : The population activation response Phi(x) (Float64) for
               input current x [Hz].
    
    N.B.: If x < -16. mA then the integrand cannot be represented
          numerically because its value is too large. Hence, Phi
          cannot be represented as well. However, since the value of
          Phi theoretically exists and is equal to zero, Phi can be
          assigned the value 0.0 for such values of x.
    =============================================================== =#
    if x >= -16.
        f(z) = exp(-x*z^2 - σ^4*z^6/48)
        integral, error = quadgk(f, -Inf, Inf, rtol=1e-8)

        # Scaling factor 1000 to get Hz
        phi = 1000/(sqrt(pi) * τ_memb * integral)
    else
        phi = 0.0
    end
    
    return phi
end

In [ ]:
function dPhi(x, σ=0.5, τ_memb=10)
    #= ===============================================================
        Compute the first derivative of Phi(I) with respect to I
    
    Args:
    x        : Input current (float)
    σ        : Standard deviation of the fluctuation
    τ_memb   : Time constant of the cell membrane
    
    Returns:
    dphi     : The Phi' evaluated at x (float)
    =============================================================== =#
    phi = Phi(x)
    if x >= -16.0
        f(z) = z^2 * exp(-x*z^2 - σ^4*z^6/48)
        integral, error = quadgk(f, -Inf,Inf, rtol=1e-8)
        # Scaling factor 1e-3 to get sec
        dphi = sqrt(pi) * τ_memb * 1e-3 * integral
        dphi = dphi * (phi^2)
    else
        dphi = 0.0    
    end
    
    return dphi
end

In [ ]:
function find_Ibias!(fun, x)
    #= ===============================================================
        Find the I_bias to apply in order to have a spontaneous
        activity of r_spont with the recurrent connection w
    
    Args:
    fun    : (multivariate) function to evaluate
    x      : variable vector (here I_ext alone)
    w      : recurrent connection parameter. DEFINED BEFORE CALL !
    r_spont: spontaneous activity value [Hz]. DEFINED BEFORE CALL !
    =============================================================== =#
    fun[1] = Phi(r_spont*w + x[1]) - r_spont
end

# 1. Spontaneous activity in 2 dimensions
Similarly to the 1D model, one looks for conditions that would give a desired (stable) spontaneous activity for each population of neurons. Considering the two-dimensional (2D) model

$$
\left\{\begin{array}{ll}
    \tau\frac{dr_P}{dt} = -r_P + \Phi [w_{PP}\cdot r_P + w_{PT}\cdot r_T + I_{ext,P} + I_{sel, P} ]\\
    \tau\frac{dr_T}{dt} = -r_T + \Phi [ w_{TP}\cdot r_P + w_{TT}\cdot r_T + I_{ext,T} + I_{sel, T} ]
\end{array}
\right.
$$

one can show that a stable spontaneous activity of $r_{spont}$ [Hz] for each population can be obtained when $I_{sel,i} = 0$ if the following conditions are met:

1. $w_{TT} = w_{PP} = w_1$, a symmetric (non)association is used $w_{TP} = w_{PT} = w_2$ and the non selective external currents are equal $I_{ext,P} = I_{ext, T}$ (sufficient condition)
    
2. $I_{ext,P} = I_{ext, T} = \Phi^{-1}(r_{spont}) - w*r_{spont}$ (similar to scenario 1 condition) with $w = w_1 + w_2$ (sufficient condition)

3. $w_1 + w_2 < \frac{1}{f}$ with $f = \Phi'(\Phi^{-1}(r_{spont}))$ (sufficient and necessary condition).

4. $w_1 - w_2 < \frac{1}{f}$ with $f = \Phi'(\Phi^{-1}(r_{spont}))$ (sufficient and necessary condition).

The two last conditions actually mean that the eigenvalues of the *Jacobian* matrix of the system must be negative (or at least their real part must be $<0$) to ensure the stability of the spontaneous activity.

In [ ]:
#=
 Considering W = [w1 w2; w2 w1]
 Determine w1 and w2 such that  -I_2x2 + diag(Phi'(Phi^-1(r_spont)))*W
 has <0 λs with r_spont the spontaneous activity vector for each
 population of neurons.

 ASSUMING r_spont_i is the same for all populations then the problem
 amounts to finding wmax = w1 + w2 = 1/Phi'(Phi^-1(r_spont_i)).

 Once wmax found, test for different w = w1 + w2. Moreover,
 w1 < (w + 1/f)/2 must be satisfied, with f = 1/Phi'(Phi^-1(r_spont_i)).
 The last inequality should imply that w2 > (w - 1/f)/2.
=#

# pars = default_pars()
# @unpack r_spont, p = pars 
p = 2 
r_spont = 5.

# Find w_max via 1/Phi'(I_{ext,w=0})
w = 0.0
I_w0 = nlsolve(find_Ibias!, [0.0]).zero[1]
f0 = dPhi(I_w0)
wmax = 1/f0
println("w1 + w2 and w1 - w2 should be less than ", wmax,
        " in order to have a stable resting state at ", r_spont,
        " Hz for each population by choosing",
        " I_ext_P = I_ext_T = Phi^-1(r_spont) - (w1+w2)*r_spont \n")

# Test for w different from w = 0
# w < wmax and w1 < (w + 1/f)/2
println("w < wmax and w1 < (w + 1/f)/2 \n")
w = wmax - 1e-3
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
f = dPhi(w*r_spont + I_spont)
w1max = (w + 1/f)/2
w1 = -w1max
w2 = w-w1

phi_spont = Phi(w1*r_spont + w2*r_spont  + I_spont)
println("(w1+w2)max = (w1-w2)max = ",wmax)
println("w1max = ", w1max)
println("w1 = ", w1)
println("w2 = ", w2)
println("I_spont = ",I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)
println("What about the stability for this FP ?")

W = [w1 w2;w2 w1]
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")


####################
# w < wmax and w1 > (w + 1/f)/2
println("w < wmax and w1 > (w + 1/f)/2 \n")
w = wmax - 1e-3
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
f = dPhi(w*r_spont + I_spont)
w1max = (w + 1/f)/2
w1 = w1max + 1e-4
w2 = w-w1

phi_spont = Phi(w1*r_spont + w2*r_spont  + I_spont)
println("(w1+w2)max = (w1-w2)max = ",wmax)
println("w1max = ", w1max)
println("w1 = ", w1)
println("w2 = ", w2)
println("I_spont = ",I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)
println("What about the stability for this FP ?")

W = [w1 w2;w2 w1]
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")


####################
# w = wmax and w1 < (w + 1/f)/2
println("w = wmax and w1 < (w + 1/f)/2 \n")
w = wmax
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
f = dPhi(w*r_spont + I_spont)
w1max = (w + 1/f)/2
w1 = -w1max
w2 = w-w1

phi_spont = Phi(w1*r_spont + w2*r_spont  + I_spont)
println("(w1+w2)max = (w1-w2)max = ",wmax)
println("w1max = ", w1max)
println("w1 = ", w1)
println("w2 = ", w2)
println("I_spont = ",I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)
println("What about the stability for this FP ?")

W = [w1 w2;w2 w1]
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")


####################
# w > wmax and w1 < (w + 1/f)/2
println("w > wmax and w1 < (w + 1/f)/2 \n")
w = wmax + 1e-3
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
f = dPhi(w*r_spont + I_spont)
w1max = (w + 1/f)/2
w1 = -w1max
w2 = w-w1

phi_spont = Phi(w1*r_spont + w2*r_spont  + I_spont)
println("(w1+w2)max = (w1-w2)max = ",wmax)
println("w1max = ", w1max)
println("w1 = ", w1)
println("w2 = ", w2)
println("I_spont = ",I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)
println("What about the stability for this FP ?")

W = [w1 w2;w2 w1]
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")


**Using synaptic connection definition from B&L**

In [ ]:
#=
 NON associated case and 1 group -> p_i = 2
 W = [JS/2 -JS(1+a)/2(1-a);-JS(1+a)/2(1-a) JS/2]

 Conditions for stability at r_spont Hz:
 0 < JS < 1/f
 0 < a < 1-JS*f 
=#

#pars = default_pars()
#@unpack r_spont, p, p_i, JE, JI = pars 
p=2;
p_i=2; 
JE=3.;
JI=JE;
r_spont=5.

# Find wmax
w = 0.0
I_w0 = nlsolve(find_Ibias!, [0.0]).zero[1]
f_rspont = dPhi(I_w0)
println("P and T nonassociated, same group (p_i = 2)")
println("JS_max = ", 1/f_rspont, "\n")


###############################
# Default value for JS from B&L
JS = 3.65 
J1 = JE + JS
println("Default JS from B&L = ", JS)


amax = 1-JS*f
if JS > 1/f_rspont # No solution can be found for a STABLE FP
    a = 0.
else
    a = amax - 1e-3
end
println("0 < a_max < 1? ", 0 < amax <= 1, "; a_max = ",amax)
println("a = ",a)
J0 = JE - JS*(a*(p_i-1)+1)/((1-a)*(p_i-1)+p-p_i)
W_nassoc = [(J1-JI)/p (J0-JI)/p; (J0-JI)/p (J1-JI)/p]

w = sum(W_nassoc[1,:]) # w1 + w2
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ", I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

f = dPhi(w*r_spont + I_spont)
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W_nassoc # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")


###############################
JS =  0.02  # 2-population adapted
J1 = JE + JS
println(" JS = ", JS)

amax = 1-JS*f
if JS > 1/f_rspont # No solution can be found for a STABLE FP
    a = 0.
else
    a = 0.02
end
println("0 < a_max < 1? ", 0 < amax <= 1, "; a_max = ",amax)
println("a = ",a)

J0 = JE - JS*(a*(p_i-1)+1)/((1-a)*(p_i-1)+p-p_i)
W_nassoc = [(J1-JI)/p (J0-JI)/p; (J0-JI)/p (J1-JI)/p]
w = sum(W_nassoc[1,:])
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ", I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

f = dPhi(w*r_spont + I_spont)
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W_nassoc # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")

In [ ]:
#=
 NON associated and P-T in NON OVERLAPPING groups -> p_i = 1
 W reduces to JS/2*[1 -1;-1 1]

 The spontaneous activity fixed point r_spont is stable 
 as long as JS < 1/f = 1/Φ'(Φ^-1(r_spont))

 Conditions:
 0 < JS < 1/f
 0 < a < 1
=#

pars = default_pars()
@unpack r_spont, p = pars 
 
println("P and T nonassociated, different groups (p_i = 1)")
println("JS_max = ", 1/f_rspont, "\n")

###################
# Default JS from B&L: > 1/Φ'(Φ^-1(r_spont))
JS = 3.65
println("JS = ", JS)

W = JS/2 * [1 -1;-1 1]
w = sum(W[1,:]) # w1 + w2
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ", I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

f = dPhi(w*r_spont + I_spont)
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")


###################
# JS < 1/f
JS = 0.02
println("JS = ", JS)

W = JS/2 * [1 -1;-1 1]
w = sum(W[1,:])
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ", I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

f = dPhi(w*r_spont + I_spont)
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")

In [ ]:
#=
 ASSOCIATED case and 1 group -> p_i = 2
 Same case as NON associated and 2 different groups

 Conditions:
 0 < JS < 1/f
 0 < a < 1
 
 If JS > 1/Φ'(Φ^-1(r_spont)), no solution can be found
 to have stable spontaneous activity at r_spont Hz
=#

pars = default_pars()
@unpack r_spont, p, p_i, JE, JI = pars 

# Find wmax
w = 0.0
I_w0 = nlsolve(find_Ibias!, [0.0]).zero[1]
f_rspont = dPhi(I_w0)
println("P and T associated, same group (p_i = 2)")
println("JS_max = ", 1/f_rspont, "\n")

###############################
# Default value for JS from B&L
JS = 3.65
J1 = JE + JS
println("Default JS = ", JS)

amax = 1.
if JS > 1/f_rspont # No solution can be found for a STABLE FP
    a = 0.
else
    a = 0.02 #amax - 1e-3
end
println("0 < a_max <= 1? ", 0 < amax <= 1, "; a_max = ",amax)
println("a = ",a)

Ja = JE + JS*(a*(p-p_i+1)-1)/((1-a)*(p_i-1)+p-p_i)
W = [(J1-JI)/p (Ja-JI)/p; (Ja-JI)/p (J1-JI)/p]
w = sum(W[1,:]) # w1 + w2
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ", I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

f = dPhi(w*r_spont + I_spont)
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")


###############################
JS =  0.02  # 2-population adapted
J1 = JE + JS
println("JS = ", JS)

amax = 1.
if JS > 1/f_rspont # No solution can be found for a STABLE FP
    a = 0.
else
    a = 0.02 #amax - 1e-3
end
println("0 < a_max <= 1? ", 0 < amax <= 1, "; a_max = ",amax)
println("a = ",a)

Ja = JE + JS*(a*(p-p_i+1)-1)/((1-a)*(p_i-1)+p-p_i)
W = [(J1-JI)/p (Ja-JI)/p; (Ja-JI)/p (J1-JI)/p]
w = sum(W[1,:])
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ", I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

f = dPhi(w*r_spont + I_spont)
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")


In [ ]:
#=
 ASSOCIATED case and 2 groups -> p_i = 1

 Conditions:
 0 < JS < 2/f
 1-1/fJS < a < 1/fJS
 
 If JS > 1/Φ'(Φ^-1(r_spont)), no solution can be found
 to have stable spontaneous activity at r_spont Hz
=#

pars = default_pars()
@unpack r_spont, p, JE, JI = pars 

p_i = 1

# Find wmax
w = 0.0
I_w0 = nlsolve(find_Ibias!, [0.0]).zero[1]
f_rspont = dPhi(I_w0)
println("P and T associated, same group (p_i = 2)")
println("JS_max = ", 2/f_rspont, "\n")

###############################
# Default value for JS from B&L
JS = 3.65
J1 = JE + JS
println("Default JS = ", JS)

amax = 1/(f_rspont*JS)
amin = 1-1/(f_rspont*JS)

if JS > 2/f_rspont # No solution can be found for a STABLE FP
    a = 0.
elseif amax > 1 # 1/f_rspont < JS < 2/f_rspont
    a = 0.5
else
    a = amax - 1e-3
end
println("amin < a_max <= 1? ", amin < amax <= 1, "; a_max = ",amax)
println("0 < amin < amax? ", 0< amin < amax , "; amin = ",amin)

println("a = ",a)

Ja = JE + JS*(a*(p-p_i+1)-1)/((1-a)*(p_i-1)+p-p_i)
W = [(J1-JI)/p (Ja-JI)/p; (Ja-JI)/p (J1-JI)/p]
w = sum(W[1,:]) # w1 + w2
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ", I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

f = dPhi(w*r_spont + I_spont)
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")

###############################
# 0 < JS < 2/f_rspont 
JS = 0.06
J1 = JE + JS
println("0 < JS < f_rspont = ", JS)

amax = 1/(f_rspont*JS)
amin = 1-1/(f_rspont*JS)

if JS > 2/f_rspont # No solution can be found for a STABLE FP
    a = 0.
elseif amax > 1 # 1/f_rspont < JS < 2/f_rspont
    a = 0.5
else
    a = amax - 1e-3
end
println("amin < a_max <= 1? ", amin < amax <= 1, "; a_max = ",amax)
println("0 < amin < amax? ", 0< amin < amax , "; amin = ",amin)

println("a = ",a)

Ja = JE + JS*(a*(p-p_i+1)-1)/((1-a)*(p_i-1)+p-p_i)
W = [(J1-JI)/p (Ja-JI)/p; (Ja-JI)/p (J1-JI)/p]
#display(W)
w = sum(W[1,:]) # w1 + w2
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ", I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

f = dPhi(w*r_spont + I_spont)
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")

###############################
# 0 < JS < 1/f_rspont 
JS = 0.02
J1 = JE + JS
println("0 < JS < 1/f_rspont = ", JS)

amax = 1/(f_rspont*JS)
amin = 1-1/(f_rspont*JS)

if JS > 2/f_rspont # No solution can be found for a STABLE FP
    a = 0.
elseif amax > 1 # 1/f_rspont < JS < 2/f_rspont
    a = 0.5
else
    a = amax - 1e-3
end
println("amin < a_max <= 1? ", amin < amax <= 1, "; a_max = ",amax)
println("0 < amin < amax? ", 0< amin < amax , "; amin = ",amin)

println("a = ",a)

Ja = JE + JS*(a*(p-p_i+1)-1)/((1-a)*(p_i-1)+p-p_i)
W = [(J1-JI)/p (Ja-JI)/p; (Ja-JI)/p (J1-JI)/p]
#display(W)
w = sum(W[1,:]) # w1 + w2
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ", I_spont)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

f = dPhi(w*r_spont + I_spont)
tmp = -Matrix(LinearAlgebra.I,p,p) + f*W # Jacobian
λmax = eigvals(tmp)
println("Greatest eigenvalue of Jacobian: ̃λmax = ", λmax[2])
println("\n")

# 2. Simulations of the temporal dynamics
In the next section, simulation of the set of ordinary differential equations (ODEs)

$$
\left\{\begin{array}{ll}
    \tau \frac{dr_P}{dt} = -r_P + \Phi(\frac{1}{p}(J_1 \cdot r_P + J_{PT} \cdot r_T) + I_{ext,P} + I_{sel,P} - I_{inh})\\
    \tau \frac{dr_T}{dt} = -r_T + \Phi(\frac{1}{p}(J_1 \cdot r_T + J_{TP} \cdot r_P) + I_{ext,T} + I_{sel,T} - I_{inh}) 
\end{array}
\right.
$$

of the firing rate of the prime and target populations is done using the following equations for the inhibitory input current and the inhibitory firing rate.
$$ I_{inh} = J_I \cdot r_I; $$

$$ r_I = \frac{1}{p} \sum_{j=1}^p r_j = \frac{1}{2}(r_P + r_T),$$

where $J_1$ is the intrapopulation efficacy, $J_I$ is the inhibitory synaptic efficacy, $J_{TP}$ or $J_{PT}$ is the synaptic efficacy from the prime to the target or vice-versa and can take values in $\{J_a; J_0\}$ that are respectively the synaptic strength between associated items and nonassociated items. $I_{ext,i}$ is the nonselective external input current for variable $i$, that is a bias current applied in order to have a spontaneous activity of $r_{spont}$ Hz and $I_{sel,i}$ is the selective external input current applird to variable $i$, that is the excitation current representing the presentation of the word.

At the end, the temporal evolution of the firing rate, i.e. $r_P(t)$ and $r_T(t)$ are obtained.

For the $2$ population scenario, the model to simulate amounts to 

$$
\left\{\begin{align*}
    \tau \frac{dr_P}{dt} &= -r_P + \Phi(\frac{(J_1-J_I)}{2} \cdot r_P + \frac{(J_{PT}-J_I)}{2} \cdot r_T + I_{ext,P} + I_{sel,P})\\ 
    &= -r_P + \Phi [w_{PP}\cdot r_P + w_{PT}\cdot r_T + I_{ext,P} + I_{sel, P}]\\
    &= -r_P + \Phi [w_1\cdot r_P + w_2\cdot r_T + I_{ext,P} + I_{sel, P}]\\
    \tau \frac{dr_T}{dt} &= -r_T + \Phi(\frac{(J_{TP}-J_I)}{2} \cdot r_P + \frac{(J_1-J_I)}{2} \cdot r_T + I_{ext,T} + I_{sel,T})\\
    &= -r_T + \Phi [ w_{TP}\cdot r_P + w_{TT}\cdot r_T + I_{ext,T} + I_{sel, T}]\\
    &= -r_T + \Phi [ w_2\cdot r_P + w_1\cdot r_T + I_{ext,T} + I_{sel, T}]
\end{align*}
\right.
$$


In [ ]:
function system!(du, u, p, t)
    τ = p[1]
    Iapp = p[2]
    W = p[3]
    I_ext = p[4]
    
    r = u

    du[:] = (-r + Phi.(W*r + I_ext + Iapp(t)))/τ
end

In [ ]:
function simulate(pars)
    #= ===============================================================
        Simulate the temporal dynamics of the prime and target
        activities.
    
    Args:
    pars    : The parameter structure
    
    Returns:
    sol     : The solution of the ODE, i.e. t = sol.t,
              r_P(t) = sol[1,:], r_T(t) = sol[2,:]
    =============================================================== =#
    # Set parameters
    @unpack τ, T, r_init = pars
    @unpack Iapp = pars
    @unpack simu_assoc = pars
    
    if simu_assoc
        I_ext = pars.I_ext_assoc * ones(pars.p)
        W = pars.W_assoc
    else
        I_ext = pars.I_ext_nassoc * ones(pars.p)
        W = pars.W_nassoc
    end
    
    p = (τ, Iapp, W, I_ext)
    u0 = r_init
    tspan = (0.0, T)
    prob = ODEProblem(system!, u0, tspan, p)
    
    # Solve
    sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8, dt=1e-5)
    
    return sol
end

In [ ]:
#=
 Example with no selective input current for P and T and no connectivity:
 I_sel_T = I_sel_P = 0.0, w1 = w2 = 0.0, r_init = [3. ;3.]
 I_ext_T = I_ext_P set to have a spontaneous activity of 5Hz

 (Valid for both associated and nonassociated cases
 and for p_i ∈ {1,2})

 The user can change T, r_init.
=#

T=50
r_init = [3.;10.] # [r_P; r_T] 

pars = default_pars()

W = zeros(pars.p, pars.p)

gr(labelfontsize=15, legendfont=13, grid=false, size=(600,400))
########################################
# Find I_ext_T = I_ext_P
w = sum(W[1,:]) # = w1 + w2
r_spont = pars.r_spont
I_ext = nlsolve(find_Ibias!, [0.0]).zero[1]
phi = Phi(w*r_spont + I_ext)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ",phi)

f = dPhi(w*r_spont+I_ext)
λ = eigvals(-Matrix(LinearAlgebra.I, pars.p, pars.p) + f*W)
println("Eigenvalues of Jacobian are ", λ)


# Set simulation parameters and solve ODE problem 
# (assoc case, p_i=2 but not important)
pars = default_pars(I_sel_T=0.0, w1=W[1,1], w2_assoc=W[1,2], W_assoc=W,
                    I_ext_assoc=I_ext, T=T, r_init=r_init)
sol = simulate(pars)
r_P = sol[1,:]
r_T = sol[2,:]
Iapp = reduce(hcat,pars.Iapp.(sol.t))

phi_P = Phi.(W[1,1]*r_P .+ W[1,2]*r_T .+ I_ext .+ Iapp[1,:])
phi_T = Phi.(W[1,2]*r_P .+ W[1,1]*r_T .+ I_ext .+ Iapp[2,:])

# Plots
plot(sol.t, sol[1,:], label=L"r_P(t)", c=:DarkBlue, lw=2)
plot!(sol.t, sol[2,:], label=L"r_T(t)", c=:Crimson, lw=2)
plot!(sol.t, phi_P, c=:DarkBlue, alpha=0.8, lw=2, ls=:dash,
      label=L"\Phi_P(I_{tot})")
plot!(sol.t, phi_T, c=:Crimson, alpha=0.8, lw=2, ls=:dashdot,
      label=L"\Phi_T(I_{tot})")

plot!(xlabel = L"t \ [msec]",
      ylabel=L"\textrm{Activity} \  r(t) \ [Hz]",
      legend=:topright, show=true, 
      thickness_scaling=1.2)

In [ ]:
#=
 Example with default JS from B&L and 2-population adapted JS and no
 input current (I_sel_T=I_sel_P=0.0).
 I_ext is set to have an ((un)stable) equilibrium at r_spont (i.e. 5Hz)
 
 ASSOCIATED P-T, 1 group -> p_i = 2

 W reduces to [JS/2 -JS/2; -JS/2 JS/2]

 The user can change r_init, T, JS
=#
T=100.
r_init = [1.; 6.]
pars = default_pars(JS=0.02)
@unpack r_spont, JS, w1, w2_assoc = pars

w1 = w1
w2 = w2_assoc

gr(labelfontsize=15, legendfont=13, grid=false, size=(600,400))
########################################
W = [w1 w2;w2 w1]
println("w1 = ",w1, "\nw2 = ", w2)

w = sum(W[1,:]) # w1 + w2
I_ext = nlsolve(find_Ibias!, [0.0]).zero[1]
phi = Phi(w*r_spont + I_ext)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ",phi)

# Set parameters
pars = default_pars(I_sel_T=0.0, w1=w1, w2_assoc=w2, I_ext_assoc=I_ext,
                    T=T, r_init=r_init, JS=JS, W_assoc=W)
f = dPhi(w*r_spont + I_ext)
λ = eigvals(-Matrix(LinearAlgebra.I, pars.p, pars.p) + f*W)
println("Eigenvalues of Jacobian are ", λ)

sol = simulate(pars)
r_P = sol[1,:]
r_T = sol[2,:]
Iapp = reduce(hcat,pars.Iapp.(sol.t))

phi_P = Phi.(w1*r_P .+ w2*r_T .+ I_ext .+ Iapp[1,:])
phi_T = Phi.(w2*r_P .+ w1*r_T .+ I_ext .+ Iapp[2,:])

# Plots
plot(sol.t, sol[1,:], label=L"r_P(t)", c=:DarkBlue, lw=2)
plot!(sol.t, sol[2,:], label=L"r_T(t)", c=:Crimson, lw=2)
plot!(sol.t, phi_P, c=:DarkBlue, alpha=0.6, lw=2, ls=:dash,
      label=L"\Phi_P(I_{tot})")
plot!(sol.t, phi_T, c=:Crimson, alpha=0.6, lw=2, ls=:dashdot, 
      label=L"\Phi_T(I_{tot})")

plot!(xlabel = L"t \ [msec]",
      ylabel=L"\textrm{Activity} \  r(t) \ [Hz]",
      legend=:right, show=true, 
      thickness_scaling=1.2)#, ylims=(-5,10))

In [ ]:
#=
 Example with default JS from B&L and 2-population adapted JS 
 and no input current (I_sel_T=I_sel_P=0.0).
 I_ext is set to have an (unstable) equilibrium at r_spont (i.e. 5Hz)
 
 NONassociated P-T, 1 group -> p_i = 2

 W = [JS/2 -JS(1+a)/2(1-a);-JS(1+a)/2(1-a) JS/2]

 Conditions for stable rspont
 0 < JS < 1/f (0.04 for r_spont=5)
 0 < a < 1-fJS

 The user can change r_init, T, JS, a
=#

T=150.
r_init = [3.; 10.]
pars = default_pars(JS=0.02, a=0.02)
@unpack r_spont, JS, w1, w2_nassoc,a = pars

w1 = w1
w2 = w2_nassoc

gr(labelfontsize=15, legendfont=13, grid=false, size=(600,400))
########################################
W = [w1 w2;w2 w1]
println("w1 = ",w1, "\nw2 = ", w2)

w = sum(W[1,:]) # w1 + w2
I_ext = nlsolve(find_Ibias!, [-2.0]).zero[1]
phi = Phi(w*r_spont + I_ext)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ",phi)

# Set parameters
pars = default_pars(I_sel_T=0.0, w1=w1, w2_nassoc=w2,
                    I_ext_nassoc=I_ext, a=a, T=T, r_init=r_init,
                    JS=JS, W_nassoc=W, simu_assoc=false)
f = dPhi(w*r_spont + I_ext)
λ = eigvals(-Matrix(LinearAlgebra.I, pars.p, pars.p) + f*W)
println("Eigenvalues of Jacobian are ", λ)

sol = simulate(pars)
r_P = sol[1,:]
r_T = sol[2,:]
Iapp = reduce(hcat,pars.Iapp.(sol.t))

phi_P = Phi.(w1*r_P .+ w2*r_T .+ I_ext .+ Iapp[1,:])
phi_T = Phi.(w2*r_P .+ w1*r_T .+ I_ext .+ Iapp[2,:])

# Plots
plot(sol.t, sol[1,:], label=L"r_P(t)", c=:DarkBlue, lw=2)
plot!(sol.t, sol[2,:], label=L"r_T(t)", c=:Crimson, lw=2)
plot!(sol.t, phi_P, c=:DarkBlue, alpha=0.6, lw=2, ls=:dash,
      label=L"\Phi_P(I_{tot})")
plot!(sol.t, phi_T, c=:Crimson, alpha=0.6, lw=2, ls=:dashdot,
      label=L"\Phi_T(I_{tot})")

plot!(xlabel = L"t \ [msec]",
      ylabel=L"\textrm{Activity} \  r(t) \ [Hz]",
      legend=:right, show=true, thickness_scaling=1.2)#, ylims=(-5,10))

In [ ]:
#=
 Example with default JS from B&L and 2-population adapted JS and
 no input current (I_sel_T=I_sel_P=0.0).
 I_ext is set to have an ((un)stable) equilibrium at r_spont (i.e. 5Hz)
 
 ASSOCIATED P-T, *2* groups -> p_i = 1

 W = [JS/2 JS(2a-1)/2;JS(2a-1)/2 JS/2]

 Conditions for stable rspont
 0 < JS < 2/f (0.08 for r_spont=5)
 1-1/fJS < a < 1/fJS

 The user can change r_init, T, JS, a
=#

T=150.
r_init = [2000.; 2500.] #[1.,6.] when JS =0.02
JS = 3.65 #0.02

pars = default_pars(JS=JS, p_i=1)
@unpack r_spont = pars


# Determine valid range for a
w=0.0
I_w0 = nlsolve(find_Ibias!, [0.0]).zero[1]
f = dPhi(I_w0)

if 0 <= JS <= 2/f
    amax = minimum([1/(f*JS),1.])
    amin = maximum([1 - 1/(f*JS), 0])
else # No solution possible for stable rspont
    amax = NaN
    amin = NaN
end
println("a_max = ", amax, " and a_min = ",amin)
if amin < amax
    a = amin + rand()*(amax-amin) #amax - 1e-3
else
    a = 0 # NO solution possible
end
println("a = ",a)


pars = default_pars(JS=JS, a=a, p_i=1)
@unpack r_spont, JS, w1, w2_assoc = pars

w1 = w1
w2 = w2_assoc

gr(labelfontsize=15, legendfont=13, grid=false, size=(600,400))
########################################
W = [w1 w2;w2 w1]
println("w1 = ",w1, "\nw2 = ", w2)

w = sum(W[1,:]) # w1 + w2
I_ext = nlsolve(find_Ibias!, [-0.0]).zero[1]
phi = Phi(w*r_spont + I_ext)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ",phi)

# Set parameters
pars = default_pars(I_sel_T=0.0, w1=w1, w2_assoc=w2,
                    I_ext_assoc=I_ext,T=T, r_init=r_init, JS=JS,
                    W_assoc=W, p_i=1, a=a)
f = dPhi(w*r_spont + I_ext)
λ = eigvals(-Matrix(LinearAlgebra.I, pars.p, pars.p) + f*W)
println("Eigenvalues of Jacobian are ", λ)

sol = simulate(pars)
r_P = sol[1,:]
r_T = sol[2,:]
Iapp = reduce(hcat,pars.Iapp.(sol.t))

phi_P = Phi.(w1*r_P .+ w2*r_T .+ I_ext .+ Iapp[1,:])
phi_T = Phi.(w2*r_P .+ w1*r_T .+ I_ext .+ Iapp[2,:])

# Plots
plot(sol.t, sol[1,:], label=L"r_P(t)", c=:DarkBlue, lw=2)
plot!(sol.t, sol[2,:], label=L"r_T(t)", c=:Crimson, lw=2)
plot!(sol.t, phi_P, c=:DarkBlue, alpha=0.6, lw=2, ls=:dash,
      label=L"\Phi_P(I_{tot})")
plot!(sol.t, phi_T, c=:Crimson, alpha=0.6, lw=2, ls=:dashdot,
      label=L"\Phi_T(I_{tot})")

plot!(xlabel = L"t \ [msec]",
      ylabel=L"\textrm{Activity} \  r(t) \ [Hz]",
      legend=:right, show=true, 
      thickness_scaling=1.2)#, ylims=(-5,10))

# 3. Phase plane analysis
## 3.1 Fixed points and nullclines

In [ ]:
interp_x = range(-2.0, 15000.0, length=190000)
interp_y = Phi.(interp_x)

Phi_inv = LinearInterpolation(interp_y, interp_x)

r_spont = 5
Phi_inv(r_spont)

In [ ]:
# Plot of inverse transfer function
r = range(1e-8, 50, length=100)
phi_inv = Phi_inv.(r)

gr(labelfontsize=13, legendfont=13, grid=false, size=(400,270))
plot(r, phi_inv, lw=2, c=:black, label="")
plot!(xlabel=L"\textrm{Average\ firing\ rate\ [Hz]}",
      ylabel=L"\textrm{Synaptic\ input\ current}")

In [ ]:
function T_nullcline(r_T, pars)
    #= ===============================================================
        Solve the nullcine ̇r_T = 0 for the given input vector r_T

    Args:
    r_T    : Firing rate of the target population (Vector of float !)
    pars   : Parameter structure

    Returns:
    r_P    : values of prime population along the nullcline r_T = 0
    =============================================================== =#
    # Set parameters
    @unpack w1, I_sel_T = pars
    @unpack simu_assoc = pars
    
    if simu_assoc
        w2 = pars.w2_assoc
        I_ext = pars.I_ext_assoc
    else
        I_ext = pars.I_ext_nassoc
        w2 = pars.w2_nassoc
    end
    # Make sure can take Phi_inv of r_T values when r_T < 1e-8
    r_T[findall(r_T .< 1e-8)] .= 1e-8 
    
    phi_inv = Phi_inv.(r_T)
    I_ext = I_ext * ones(length(r_T))
    I_sel_T = I_sel_T * ones(length(r_T))
    
    r_P = (phi_inv - w1*r_T - (I_ext + I_sel_T))/w2
    
    return r_P   
end


function P_nullcline(r_P, pars)
    #= ===============================================================
        Solve the nullcine ̇r_P = 0 for the given input vector r_P

    Args:
    r_P    : Firing rate of the prime population (Vector of float !)
    pars   : Parameter structure

    Returns:
    r_T    : values of prime population along the nullcline r_T = 0
    =============================================================== =#
    # Set parameters
    @unpack w1, I_sel_P = pars
    @unpack simu_assoc = pars
    
    if simu_assoc
        w2 = pars.w2_assoc
        I_ext = pars.I_ext_assoc
    else
        I_ext = pars.I_ext_nassoc
        w2 = pars.w2_nassoc
    end
    # Make sure can take Phi_inv of r_T values when r_T = 0.0
    r_P[findall(r_P .< 1e-8)] .= 1e-8 
    
    phi_inv = Phi_inv.(r_P)
    I_ext = I_ext * ones(length(r_P))
    I_sel_P = I_sel_P * ones(length(r_P))
    
    r_T = (phi_inv - w1*r_P - (I_ext + I_sel_P))/w2
    
    return r_T    
end

In [ ]:
function plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
    #= ===============================================================
        Plot the nullclines for the input vector r_T and r_P
        Note that a (blank) plot has to be defined beforehand!
        The structure parameter is given to solve the problem of
        ∞ # of solutions when r_{T,P} = 0.0
    =============================================================== =#
    plot!(r_T, T_nullc, c=:Crimson, label=L"\dot{r}_T = 0", lw=2)
    plot!(P_nullc, r_P, c=:darkblue, label=L"\dot{r}_P = 0", lw=2)
    plot!(xlabel=L"r_T\ [Hz]", ylabel=L"r_P\ [Hz]")

    # Solve the problem of ∞ # of solutions when r_{T,P} = 0.0
    if pars.simu_assoc
        w2 = pars.w2_assoc
    else
        w2 = pars.w2_nassoc
    end
    
    if r_P[1] <= 1e-5 
        if w2 <= 0 # replicate P_nullc[1] to the right (growing r_T)
            plot!(P_nullc[1] .+ r_T, r_P[1]*ones(length(r_T)), lw=2,
                  c=:darkblue, label="")
        else
            plot!(P_nullc[1] .- r_T, r_P[1]*ones(length(r_T)), lw=2,
                  c=:darkblue, label="")
        end
    end
        
    if r_T[1] <= 1e-5
        if w2 <= 0 # replicate T_nullc[1] upwards (growing r_P)
            plot!(r_T[1]*ones(length(r_P)), T_nullc[1] .+ r_P, lw=2,
                  c=:crimson, label="")
        else
           plot!(r_T[1]*ones(length(r_P)), T_nullc[1] .- r_P, lw=2,
                 c=:crimson, label="")
        end
    end
end

In [ ]:
function plot_vector_field(r_P, r_T, pars, skip, arrowlength=1.)
    #= ===============================================================
        Plot the vectorfield for the input vector r_T and r_P and the 
        parameter structure pars.
        Note that a (blank) plot has to be defined beforehand!
    
    Args:
    skip        : Integer, vector field at every 'skip' values of
                  r_T and r_P.
    arrowlength : Control length of arrow in vector field (∈ ]0,1])
    
    Returns nothing
    =============================================================== =#
    @unpack w1, I_sel_P, I_sel_T = pars
    @unpack simu_assoc, τ = pars

    if simu_assoc
        w2 = pars.w2_assoc
        I_ext = pars.I_ext_assoc
    else
        I_ext = pars.I_ext_nassoc
        w2 = pars.w2_nassoc
    end

    # Vector field as such
    u(x,y) = (-x + Phi(w2*y + w1*x + I_ext + I_sel_T))/τ # ̇r_T
    v(x,y) = (-y + Phi(w1*y + w2*x + I_ext + I_sel_P))/τ # ̇r_P
    df(x,y) = (arrowlength*u(x,y), arrowlength*v(x,y))
    
    # Evaluating points
    r_T_vect_field = r_T[1:skip:length(r_T)]
    r_P_vect_field = r_P[1:skip:length(r_P)]

    r_TT = [x for x in r_T_vect_field for y in r_P_vect_field]
    r_PP = [y for x in r_T_vect_field for y in r_P_vect_field]
    quiver!(r_TT, r_PP, quiver=df, c=:gray70)
end

In [ ]:
function plot_trajectory(r_P0, r_T0, pars, colour=:green)
    #= ===============================================================
        Plot the trajectory starting from [r_P0, r_T0] and with
        parameter structure pars.
        The user can choose the colour of the trajectory, default is
        green.
        Note that a (blank) plot has to be defined beforehand.
    =============================================================== =#
    r_0 = [r_P0, r_T0]

    # Set parameters
    @unpack τ, T = pars
    @unpack Iapp = pars
    @unpack simu_assoc = pars
    
    if simu_assoc
        I_ext = pars.I_ext_assoc * ones(pars.p)
        W = pars.W_assoc
    else
        I_ext = pars.I_ext_nassoc * ones(pars.p)
        W = pars.W_nassoc
    end
    
    p = (τ, Iapp, W, I_ext)
    u0 = r_0
    tspan = (0.0, T)
    prob = ODEProblem(system!, u0, tspan, p)
    
    # Solve
    sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8, dt=1e-5)

    plot!(sol[2,:], sol[1,:], lw=2, c=colour,
          label=string(latexstring("[r_P(0), r_T(0)] = [$(Float16(r_0[1])), $(Float16(r_0[2]))]")))
    scatter!([sol[2,1]], [sol[1,1]], c=colour, ms=5, msc=colour,
             label="")
end

In [ ]:
# Simple test, JS = a = 0.02, I_sel_{T,P} = 0.0,
# I_ext set for r_spont = 5 Hz

pars= default_pars(I_sel_T=0.0)
stop = 10.
r_T = convert(Vector,range(0.0, stop, length=50))
r_P = convert(Vector,range(0.0, stop, length=50))

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)
    

# Visualize
gr(legendfontsize=10, labelfontsize=13, grid=false, size=(600,400))
    
plt = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)

plot_vector_field(r_P, r_T, pars, 4, 1.0)


plot!(xlim=(r_T[1], r_T[end]),ylim=(r_P[1], r_P[end]))

# Sample trajectories
r_P0 = 8.0
r_T0 = 2.0
plot_trajectory(r_P0, r_T0, pars, :purple)

r_P0 = 3.0
r_T0 = 1.0
plot_trajectory(r_P0, r_T0, pars, :cornflowerblue)

r_P0 = 1.0
r_T0 = 5.0
plot_trajectory(r_P0, r_T0, pars, :orange)

r_P0 = 4.0
r_T0 = 8.0
plot_trajectory(r_P0, r_T0, pars, :forestgreen)

r_P0 = 6.0
r_T0 = 9.0
plot_trajectory(r_P0, r_T0, pars, :gold)

r_P0 = 9.0
r_T0 = 6.0
plot_trajectory(r_P0, r_T0, pars, :deeppink)

plot!(legend=:outerright)

### 3.1.1. Associated P-T within one group

In [ ]:
#=
 Example with JS = 0.02 (2-population adapted),
 (a = 0.02), I_ext set for FP at r_spont = 5 Hz

 I_sel can vary

 The user can change stopP, stopT, len, skip, arrowlength,
 I_sel_T and I_sel_P.
=#

stopP = 40.
stopT = 20.
len = 50

skip = 5
arrowlength=0.8
r_T = convert(Vector,range(0.0, stopT, length=len))
r_P = convert(Vector,range(0.0, stopP, length=len))

gr(legendfontsize=10, labelfontsize=15, titlefontsize=13,
    grid=false, size=(900,900))
################################
# I_sel_T = I_sel_P = 0.0
pars= default_pars(I_sel_T=0.0)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt22 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 0.0; I_sel_P = 0.2
pars= default_pars(I_sel_T=0.0, I_sel_P=0.2)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt32 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 0.2; I_sel_P = 0.0
pars= default_pars(I_sel_T=0.2, I_sel_P=0.0)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt23 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 0.2; I_sel_P = 0.2
pars= default_pars(I_sel_T=0.2, I_sel_P=0.2)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt33 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 0.0; I_sel_P = -0.2
pars= default_pars(I_sel_T=0.0, I_sel_P=-0.2)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt12 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 0.2; I_sel_P = -0.2
pars= default_pars(I_sel_T=0.2, I_sel_P=-0.2)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt13 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = -0.2; I_sel_P = -0.2
pars= default_pars(I_sel_T=-0.2, I_sel_P=-0.2)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt11 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = -0.2; I_sel_P = -0.2
pars= default_pars(I_sel_T=-0.2, I_sel_P=0.0)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt21 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = -0.2; I_sel_P = 0.2
pars= default_pars(I_sel_T=-0.2, I_sel_P=0.2)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt31 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))



plot(plt11, plt12, plt13, plt21, plt22, plt23, plt31, plt32, plt33,
     layout=(3,3))
plot!(xlim=(0.0, r_T[end]),ylim=(0.0, r_P[end]), margin=10px)

In [ ]:
#=
 Example with JS = 3.65 (default B&L), (a = 0.02) 

 I_sel can vary

 The user can change r_0 for sample trajectories, I_sel_T,
 I_sel_P and JS 
=#

JS = 3.65

stopP = 2000.
stopT = 2000.
len = 1000

skip = 100
arrowlength=0.8
r_T = convert(Vector,range(0.0, stopT, length=len))
r_P = convert(Vector,range(0.0, stopP, length=len))

gr(legendfontsize=10, labelfontsize=15, titlefontsize=14,
    grid=false, size=(900,900))
################################
# I_sel_T = I_sel_P = 0.0
pars= default_pars(I_sel_T=0.0, JS=JS)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt22 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 0., I_sel_P = 100.
pars= default_pars(I_sel_T=0.0, I_sel_P=100., JS=JS)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt32 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 100., I_sel_P = 0.
pars= default_pars(I_sel_T=100.0, I_sel_P=0., JS=JS)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt23 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 100., I_sel_P = 100.
pars= default_pars(I_sel_T=100.0, I_sel_P=100., JS=JS)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt33 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 0., I_sel_P = -100.
pars= default_pars(I_sel_T=0.0, I_sel_P=-100., JS=JS)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt12 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 100., I_sel_P = -100.
pars= default_pars(I_sel_T=100.0, I_sel_P=-100., JS=JS)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt13 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = -100., I_sel_P = -100.
pars= default_pars(I_sel_T=-100.0, I_sel_P=-100., JS=JS)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt11 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = -100., I_sel_P = 0.
pars= default_pars(I_sel_T=-100.0, I_sel_P=0., JS=JS)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt21 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = -100., I_sel_P = 100.
pars= default_pars(I_sel_T=-100.0, I_sel_P=100., JS=JS)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt31 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))



plot(plt11, plt12, plt13, plt21, plt22, plt23, plt31, plt32, plt33,
     layout=(3,3))
plot!(xlim=(-100.0, r_T[end]),ylim=(-100.0, r_P[end]), margin=10px)

### 3.1.2. Nonassociated P-T within one group
When P and T are nonassociated within the same group ($\rightarrow p_i = 2$), the equilibrium for the spontaneous activity at $r_{spont}$ Hz is stable if $0 < J_S < 1/f$ (with $f = \Phi’(\Phi^{-1} (r_{spont}))$) and $0 < a < 1-f\cdot J_S$

In [ ]:
pars = default_pars()
@unpack r_spont, p, p_i = pars 

w = 0.0
I_w0 = nlsolve(find_Ibias!, [0.0]).zero[1]
f = dPhi(I_w0)
println("P and T nonassociated, same group (p_i = 2)")
println("JS_max = ", 1/f, "\n")

phi_spont = Phi(w*r_spont + I_w0)
println("Phi(w1*r_spont + w2*r_spont + I_spont) = ", phi_spont)

In [ ]:
#=
 Example with JS = {3.65, 0.02}, a = {0.02,0.06, 0.99}, 
 I_ext set for FP at r_spont = 5 Hz

 I_sel can vary

 The user can change I_sel_T (> 0), I_sel_P (>0), JS, a, stopP, stopT,
 len, skip and arrowlength.
=#

JS = 0.02
a = 0.02
I_sel_T = 0.3#100. #0.2 
I_sel_P = 0.2#100. #0.2

stopP = 25 #2000.
stopT = 25 #2000.
len = 100 #500 #10000

skip = 10 #50 #1200
arrowlength=0.5

gr(legendfontsize=10, labelfontsize=15, titlefontsize=14,
    grid=false, size=(900,900))
######################################################################
amax = (1-JS*f)
println("0 < a_max < 1? ", 0 < amax <= 1, "; a_max = ",amax)
println("a = ",a)

w1 = JS/2
w2 = -JS*(a+1)/(2 * (1-a))
W_nassoc = [w1 w2;w2 w1]

w = sum(W_nassoc[1,:]) # w1 + w2
I_ext = nlsolve(find_Ibias!, [10.0]).zero[1]
phi_spont = Phi(w*r_spont + I_ext)
println("I_ext = ", I_ext)
println("Phi(w1*r_spont + w2*r_spont + I_ext) = ", phi_spont)

r_T = convert(Vector,range(0.0, stopT, length=len))
r_P = convert(Vector,range(0.0, stopP, length=len))


################################
# I_sel_T = I_sel_P = 0.0
pars= default_pars(I_sel_T=0.0, JS=JS, a=a, w1=w1, w2_nassoc=w2,
                   I_ext_nassoc=I_ext, W_nassoc=W_nassoc,
                   simu_assoc=false)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt22 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=
      latexstring("I_{sel,P} = $(pars.I_sel_P);I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 0., I_sel_P >0.
pars= default_pars(I_sel_T=0.0, I_sel_P=I_sel_P, JS=JS, a=a, w1=w1,
                   w2_nassoc=w2,I_ext_nassoc=I_ext, W_nassoc=W_nassoc,
                   simu_assoc=false)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt32 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = >0, I_sel_P = 0.
pars= default_pars(I_sel_T=I_sel_T, I_sel_P=0., JS=JS, a=a, w1=w1,
                   w2_nassoc=w2, I_ext_nassoc=I_ext, W_nassoc=W_nassoc,
                   simu_assoc=false)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt23 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=
      latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = >0., I_sel_P = >0.
pars= default_pars(I_sel_T=I_sel_T, I_sel_P=I_sel_P, JS=JS, a=a, w1=w1,
                   w2_nassoc=w2, I_ext_nassoc=I_ext, W_nassoc=W_nassoc,
                   simu_assoc=false)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt33 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=
      latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = 0., I_sel_P = <0.
pars= default_pars(I_sel_T=0.0, I_sel_P=-I_sel_P, JS=JS, a=a, w1=w1,
                   w2_nassoc=w2, I_ext_nassoc=I_ext, W_nassoc=W_nassoc,
                   simu_assoc=false)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt12 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=
      latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = >0., I_sel_P = < 0.
pars= default_pars(I_sel_T=I_sel_T, I_sel_P=-I_sel_P, JS=JS, a=a,
                   w1=w1, w2_nassoc=w2, I_ext_nassoc=I_ext, 
                   W_nassoc=W_nassoc, simu_assoc=false)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt13 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=
      latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = <0., I_sel_P = <0.
pars= default_pars(I_sel_T=-I_sel_T, I_sel_P=-I_sel_P, JS=JS, a=a, 
                   w1=w1, w2_nassoc=w2, I_ext_nassoc=I_ext, 
                   W_nassoc=W_nassoc, simu_assoc=false)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
plt11 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = -100., I_sel_P = 0.
pars= default_pars(I_sel_T=-I_sel_T, I_sel_P=0., JS=JS, a=a, w1=w1,
                   w2_nassoc=w2, I_ext_nassoc=I_ext, 
                   W_nassoc=W_nassoc, simu_assoc=false)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
gr(legendfontsize=10, labelfontsize=15)
plt21 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=
      latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))


################################
# I_sel_T = -100., I_sel_P = 100.
pars= default_pars(I_sel_T=-I_sel_T, I_sel_P=I_sel_P, JS=JS, a=a,
                   w1=w1, w2_nassoc=w2, I_ext_nassoc=I_ext, 
                   W_nassoc=W_nassoc, simu_assoc=false)

# Compute nullclines
T_nullc = T_nullcline(r_T, pars)
P_nullc = P_nullcline(r_P, pars)

# Visualize
gr(legendfontsize=10, labelfontsize=15)
plt31 = plot()
plot_nullclines(r_P, r_T, P_nullc, T_nullc, pars)
plot_vector_field(r_P, r_T, pars, skip, arrowlength)
plot!(title=
      latexstring("I_{sel,P} = $(pars.I_sel_P); I_{sel,T} = $(pars.I_sel_T)"))



plot(plt11, plt12, plt13, plt21, plt22, plt23, plt31, plt32, plt33,
     layout=(3,3))
#plot!(xlim=(-100.0, r_T[end]),ylim=(-100.0, r_P[end]), margin=10px)
plot!(xlim=(r_T[1], r_T[end]),ylim=(r_P[1], r_P[end]), margin=10px)
plot!(plot_title=latexstring("J_S = $(JS);a = $(a)"))